<a href="https://colab.research.google.com/github/NoumanAhmad448/Improving_Detection_of_Person_using_dense_pooling/blob/master/Improving_Detection_of_Person_using_dense_pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import numpy as np
import matplotlib.pyplot as plt
import requests

from sklearn.model_selection import train_test_split
from numpy import random
from torch import nn
try:
  from torchsummary import summary
except:
  !pip install torchsummary
  from torchsummary import summary

try:
  from torchmetrics import Accuracy
except:
  !pip install torchmetrics
  from torchmetrics import Accuracy

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

#new 
from torchvision import models
from torchvision import transforms
from torchsummary import summary

try:
  import fiftyone as fo
except:
  !pip install fiftyone
  import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import fiftyone.utils.coco as fouc
from PIL import Image
import pandas as pd
from torchvision.transforms import functional as func
import fiftyone.brain as fob
import eta.core.utils as etau
from pathlib import Path
import sys
from google.colab import files,drive
import fiftyone.utils.random as four
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
import fiftyone.utils.annotations as foua
import cv2
import os,distutils

try:
  import detectron2
except:
  # !git clone 'https://github.com/facebookresearch/detectron2'
  # dist = distutils.core.run_setup("./detectron2/setup.py")
  # !python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
  # sys.path.insert(0, os.path.abspath('./detectron2'))
  !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
  import detectron2

from google.colab.patches import cv2_imshow
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
# auth.authenticate_user()
!pip install "umap-learn>=0.5"
!pip install pyyaml==5.1
from detectron2.engine import DefaultTrainer
import json
from detectron2.structures import BoxMode
from detectron2.modeling import build_model


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 11.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.7 MB/s eta 0:00:0

INFO:fiftyone.migrations.runner:Migrating database to v0.20.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-5ogdxe6u
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-5ogdxe6u
  Resolved https://github.com/facebookresearch/detectron2.git to commit 2c6c380f94a27bd8455a39506c9105f652b9f760
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filen

In [2]:
'''
download the helper functions
'''
helper_file = "helper_funs.py"
field_file = "field_names.py"

helper_functions = requests.get("https://raw.githubusercontent.com/NoumanAhmad448/deep_learning_pytorch_python_computer_vision/master/helper_funs.py")
with open(helper_file, "wb") as module:
  module.write(helper_functions.content)

from helper_funs import *
# ******************************************
helper_functions = requests.get("https://raw.githubusercontent.com/NoumanAhmad448/deep_learning_pytorch_python_computer_vision/master/field_names.py")
with open(field_file, "wb") as module:
  module.write(helper_functions.content)

from field_names import *
# ***************
dense_file = "dense_file.py"
dense_pooling_fun = requests.get("https://raw.githubusercontent.com/NoumanAhmad448/deep_learning_pytorch_python_computer_vision/master/dense_pooling.py")
with open(dense_file, "wb") as module:
  module.write(dense_pooling_fun.content)

from dense_file import dense_pooling

In [ ]:
# Helpful resource to download COCO dataset
# it will download the images in the director mentioned below and according to COCO dataset
# resource, currently it is recommended way to download the COCO dataset

dataset = fo.Dataset.from_dir("gdrive/MyDrive/data/coco-17/person",fo.types.FiftyOneDataset,shuffle=True,seed=51,
                               name="coco-178"),split="train",class=["person"])
print(dataset)

In [ ]:
# exporting dataset
# custom function 
export_dataset(file_name="coco-17/person",dataset=dataset,
               dataset_type=fo.types.COCODetectionDataset)

Mounted at /content/gdrive/
 100% |███████████████| 7000/7000 [2.3m elapsed, 0s remaining, 63.4 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 7000/7000 [2.3m elapsed, 0s remaining, 63.4 samples/s]      


In [ ]:
# exporting labeling
dataset.export(
    # labels_path="gdrive/MyDrive/data/coco-17/person/coco.json",
    labels_path="person/coco.json",
    dataset_type=fo.types.COCODetectionDataset,
    label_field="ground_truth")

 100% |███████████████| 7000/7000 [36.1s elapsed, 0s remaining, 251.9 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 7000/7000 [36.1s elapsed, 0s remaining, 251.9 samples/s]      


In [3]:
# You may use the functionality to load dataset directly from Google Drive
mount_google_drive()

Mounted at /content/gdrive/


In [28]:
dataset_name = "coco_train5"
model_config = "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"
ds_path="gdrive/MyDrive/data/coco-17/person/"
output_dir = "output_17_1"

In [29]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(model_config))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# to train uncomment the below 
# to test comment
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_config)


cfg.DATASETS.TRAIN = (dataset_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 100
cfg.SOLVER.STEPS = [] 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 91
cfg.MODEL.DEVICE= setup_device()
cfg.OUTPUT_DIR = ds_path+output_dir
# to train comment the below 
# to test uncomment
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# to train uncomment the below 
# to test comment
# dense_pooling(cfg)

In [14]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances(dataset_name, {}, ds_path+"coco.json", ds_path+"data")

In [12]:
dataset_dicts = DatasetCatalog.get(dataset_name)

WARNING [05/19 13:00:20 d2.data.datasets.coco]: gdrive/MyDrive/data/coco-17/person/coco.json contains 66502 annotations, but only 66349 of them match to images in the file.
[05/19 13:00:20 d2.data.datasets.coco]: Loaded 6982 images in COCO format from gdrive/MyDrive/data/coco-17/person/coco.json


In [21]:
#train model
trainer.train()

[05/19 13:20:56 d2.engine.train_loop]: Starting training from iteration 0
[05/19 13:21:06 d2.utils.events]:  eta: 0:00:35  iter: 19  total_loss: 5.336  loss_cls: 4.315  loss_box_reg: 0.9336  loss_rpn_cls: 0.02965  loss_rpn_loc: 0.06519    time: 0.4358  last_time: 0.3428  data_time: 0.0392  last_data_time: 0.0058   lr: 0.00019081  max_mem: 3200M
[05/19 13:21:16 d2.utils.events]:  eta: 0:00:26  iter: 39  total_loss: 2.727  loss_cls: 1.644  loss_box_reg: 0.8835  loss_rpn_cls: 0.0433  loss_rpn_loc: 0.09041    time: 0.4813  last_time: 0.4375  data_time: 0.0200  last_data_time: 0.0061   lr: 0.00039061  max_mem: 3200M
[05/19 13:21:25 d2.utils.events]:  eta: 0:00:17  iter: 59  total_loss: 2.268  loss_cls: 1.249  loss_box_reg: 0.8442  loss_rpn_cls: 0.03613  loss_rpn_loc: 0.07718    time: 0.4591  last_time: 0.3416  data_time: 0.0107  last_data_time: 0.0242   lr: 0.00059041  max_mem: 3202M
[05/19 13:21:34 d2.utils.events]:  eta: 0:00:08  iter: 79  total_loss: 2.05  loss_cls: 1.111  loss_box_reg: 

In [ ]:
# model training
os.makedirs(ds_path+output_dir, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)


In [31]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Prediction Without Dense Pooling
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, dataset_name)
results = inference_on_dataset(predictor.model, val_loader, evaluator)



[05/19 13:25:40 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from gdrive/MyDrive/data/coco-17/person/output_17_1/model_final.pth ...
WARNING [05/19 13:25:42 d2.data.datasets.coco]: gdrive/MyDrive/data/coco-17/person/coco.json contains 66502 annotations, but only 66349 of them match to images in the file.
[05/19 13:25:42 d2.data.datasets.coco]: Loaded 6982 images in COCO format from gdrive/MyDrive/data/coco-17/person/coco.json
[05/19 13:25:43 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/19 13:25:43 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[05/19 13:25:43 d2.data.common]: Serializing 6982 elements to byte tensors and concatenating them all ...
[05/19 13:25:43 d2.data.common]: Serialized dataset takes 5.15 MiB
[05/19 13:25:43 d2.evaluation.evaluator]: Start inference on 6982 batches
[0

In [9]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#Prediction With Dense Layering

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, dataset_name)
results = inference_on_dataset(predictor.model, val_loader, evaluator)

[05/19 12:00:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from gdrive/MyDrive/data/coco-17/person/output_17_1/model_final.pth ...
[05/19 12:00:03 d2.data.datasets.coco]: Loading gdrive/MyDrive/data/coco-17/person/coco.json takes 1.16 seconds.
WARNING [05/19 12:00:03 d2.data.datasets.coco]: gdrive/MyDrive/data/coco-17/person/coco.json contains 66502 annotations, but only 66349 of them match to images in the file.
[05/19 12:00:03 d2.data.datasets.coco]: Loaded 6982 images in COCO format from gdrive/MyDrive/data/coco-17/person/coco.json
[05/19 12:00:04 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/19 12:00:04 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[05/19 12:00:04 d2.data.common]: Serializing 6982 elements to byte tensors and concatenating them all ...
[05/19 12:00:04 d2.data.common

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/19 12:00:31 d2.evaluation.evaluator]: Inference done 16/6982. Dataloading: 0.2881 s/iter. Inference: 0.1094 s/iter. Eval: 0.0003 s/iter. Total: 0.3979 s/iter. ETA=0:46:11
[05/19 12:00:37 d2.evaluation.evaluator]: Inference done 30/6982. Dataloading: 0.2856 s/iter. Inference: 0.1025 s/iter. Eval: 0.0003 s/iter. Total: 0.3884 s/iter. ETA=0:45:00
[05/19 12:00:42 d2.evaluation.evaluator]: Inference done 44/6982. Dataloading: 0.2859 s/iter. Inference: 0.0996 s/iter. Eval: 0.0003 s/iter. Total: 0.3859 s/iter. ETA=0:44:37
[05/19 12:00:48 d2.evaluation.evaluator]: Inference done 57/6982. Dataloading: 0.2976 s/iter. Inference: 0.0998 s/iter. Eval: 0.0003 s/iter. Total: 0.3978 s/iter. ETA=0:45:54
[05/19 12:00:53 d2.evaluation.evaluator]: Inference done 71/6982. Dataloading: 0.2987 s/iter. Inference: 0.0993 s/iter. Eval: 0.0003 s/iter. Total: 0.3984 s/iter. ETA=0:45:53
[05/19 12:00:59 d2.evaluation.evaluator]: Inference done 85/6982. Dataloading: 0.2926 s/iter. Inference: 0.1032 s/iter. Eval: